In [1]:
#connect to camera
from azure.storage.blob import BlobServiceClient
from PIL import Image
import cv2

In [2]:

import numpy as np

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frames.append(frame)

    cap.release()

    return frames

In [3]:
def upload_blob(data, filename):
    try:
        # Connect to the Azure storage account
        connect_str = "DefaultEndpointsProtocol=https;AccountName=praetorianblob;AccountKey=PEv0rg+hl5Y2j4I52M6eddkBVK/1JghLJj95PfBtqC8F/T3bjKgfTwxxi6kGGPIvonrSLvvkI9Vc+AStNfJI6w==;EndpointSuffix=core.windows.net"
        blob_client = BlobServiceClient.from_connection_string(connect_str)
        print("\nUploading to Azure Storage as blob:\n\t" + filename)
        # Upload the created file
        blob_client = blob_client.get_blob_client(container="onpremvideo", blob=filename)
        blob_client.upload_blob(data)
            
    except Exception as ex:
        print('Exception:')
        print(ex)

In [4]:

def extract_frames(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames


In [5]:
import datetime

def run_camera(model, preprocessor):
    # Connect to the camera
    cap = cv2.VideoCapture(0)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    # 5 seconds of video
    seconds = 5

    # Get the current time
    now = datetime.datetime.now()

    # Create a filename
    filename = now.strftime("%Y-%m-%d-%H-%M-%S") + ".avi"

    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(filename, fourcc, 20.0, (640, 480))

    # Get the end time
    end = now + datetime.timedelta(seconds=seconds)

    # Loop until the end time
    while datetime.datetime.now() < end:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:
            # Write the frame to the video
            out.write(frame)

            # Display the resulting frame
            cv2.imshow('Frame', frame)

            # Press Q on the keyboard to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    # Release the video capture
    cap.release()

    # Release the video writer
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows()

    # Extract the frames
    frames = extract_frames(filename)

    # Check if frames are extracted successfully
    if not frames:
        print("Error extracting frames from the video")
        return

    # Loop through the frames
    for frame in frames:
        # Preprocess the frame
        preprocessed_frame = preprocessor(frame)

        # Check if frame preprocessing is successful
        if preprocessed_frame is None:
            print("Error preprocessing frame")
            continue  # Skip to the next iteration

        # Make a prediction
        prediction = model.predict(preprocessed_frame)

        # Get the prediction label
        label = prediction.argmax(axis=-1)

        # Get the prediction label text
        label_text = model.get_label_name(label)

        # Draw the label on the frame
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

        # Display the frame
        cv2.imshow('Frame', frame)

        # Press Q on the keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Closes all the frames
    cv2.destroyAllWindows()

    # Upload the video to Azure Blob Storage
    upload_blob(open(filename, "rb"), filename)

    # Delete the video file
    # os.remove(filename)


In [6]:
from transformers import YolosImageProcessor, YolosForObjectDetection

/Users/salim/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/salim/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [7]:
model = YolosForObjectDetection.from_pretrained("/Users/salim/dev/prototype/praetorian/onpremise/service/yolos-tiny")
preprocessor = YolosImageProcessor.from_pretrained("/Users/salim/dev/prototype/praetorian/onpremise/service/yolos-tiny")

In [8]:
run_camera(model, preprocessor)

[ WARN:0@8.595] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@10.122] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements


Error opening video file
Error extracting frames from the video


[ WARN:0@15.202] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (1127) open OpenCV | GStreamer warning: Error opening bin: unexpected reference "2023-12-30-17-02-24" - ignoring
[ WARN:0@15.203] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
OpenCV: Couldn't read video stream from file "2023-12-30-17-02-24.avi"
